In [15]:
import torch
from torch import nn, optim
from torchvision import datasets
from torchvision.transforms import v2

In [39]:
# transform = v2.Compose(
#     [
#         v2.ToTensor(),
#         v2.Normalize((0.5,), (0.5,))
#     ]
# )

transforms = v2.Compose([
    # v2.RandomResizedCrop(size=(28, 28), antialias=True),
    # v2.RandomHorizontalFlip(p=0.5),
    v2.ToImage(),  # Преобразование в изображение
    v2.ToDtype(torch.float32, scale=True),  # Преобразование в тензор с масштабированием
    v2.Normalize((0.5,), (0.5,))
])
mnist = datasets.MNIST(root='./data', train=True, download=True, transform=transforms)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=transforms)

In [40]:
train_loader = torch.utils.data.DataLoader(mnist, batch_size=1000, shuffle=True)
test_loader = torch.utils.data.DataLoader(mnist_testset, batch_size=1000, shuffle=False)

In [45]:
class MNISTClassification(nn.Module):
    def __init__(self):
        super(MNISTClassification, self).__init__()
        self.linear = nn.Linear(28*28, 10)
    
    def forward(self, image):
        out = image.view(-1, 28*28)
        out = self.linear(out)
        return out
    
mnist_model = MNISTClassification()
criterion = nn.CrossEntropyLoss()
optimizer  = optim.SGD(mnist_model.parameters(), lr=0.05)

In [46]:
from tqdm import tqdm

num_epochs = 20
for epoch in range(num_epochs):
    mnist_model.train()
    with tqdm(total=len(train_loader), desc=f'Epoch [{epoch+1:>2}/{num_epochs}]') as pbar:
        for images, labels in train_loader:
            optimizer.zero_grad()
            outputs = mnist_model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            pbar.set_postfix({'mnist loss': f'{loss.item():.4f}'})
            pbar.update(1)

mnist_model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = mnist_model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Epoch [20/20]: 100%|██████████| 60/60 [00:24<00:00,  2.48it/s, mnist loss=0.2716]


Accuracy of the model on the test images: 91.63%
